In [10]:
import yfinance as yf

token = 'BTC-USD'
# start_date = '2015-01-01'

data = yf.download(token, interval='1d')
data = data.droplevel('Ticker', axis=1)
data.to_csv('data/BTC-USD.csv')
data

[*********************100%***********************]  1 of 1 completed


Price              Close          High           Low          Open  \
Date                                                                 
2014-09-17    457.334015    468.174011    452.421997    465.864014   
2014-09-18    424.440002    456.859985    413.104004    456.859985   
2014-09-19    394.795990    427.834991    384.532013    424.102997   
2014-09-20    408.903992    423.295990    389.882996    394.673004   
2014-09-21    398.821014    412.425995    393.181000    408.084991   
...                  ...           ...           ...           ...   
2025-03-22  83832.484375  84513.875000  83674.781250  84046.257812   
2025-03-23  86054.375000  86094.781250  83794.914062  83831.898438   
2025-03-24  87498.914062  88758.726562  85541.195312  86070.929688   
2025-03-25  87471.703125  88542.398438  86346.078125  87512.820312   
2025-03-26  86587.390625  88254.101562  86504.062500  87458.625000   

Price            Volume  
Date                     
2014-09-17     21056800  
2014-09-18     34483200  
2014-09-19     37919700  
2014-09-20     36863600  
2014-09-21     26580100  
...                 ...  
2025-03-22   9863214091  
2025-03-23  12594615537  
2025-03-24  34582604933  
2025-03-25  30005840049  
2025-03-26  28536555520  

[3844 rows x 5 columns]

In [8]:
import pandas as pd
import talib

data['ema20'] = talib.EMA(data['Close'], timeperiod=20)
data['ema50'] = talib.EMA(data['Close'], timeperiod=50)
data['rsi10'] = talib.RSI(data['Close'], timeperiod=10)
data['rsi30'] = talib.RSI(data['Close'], timeperiod=30)
data['rsi200'] = talib.RSI(data['Close'], timeperiod=200)

data['macd'], data['signal'], data['hist'] = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

data['Price-Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)
data.dropna(inplace=True)
data.to_csv('/data/BTC-I.csv')
data

OSError: Cannot save file into a non-existent directory: '\data'

# Stream_Crypto_Data 

In [4]:
import json 
import websocket
import pandas as pd
import numpy as np

coin = 'btcusdt@kline_1m'
socket = 'wss://stream.binance.com:9443/stream?streams=' + coin
print(socket)


wss://stream.binance.com:9443/stream?streams=btcusdt@kline_1m


In [5]:
def extract_ohlcv(source):
    event_time = pd.to_datetime(source['data']['E'], unit='ms')
    open = source['data']['k']['o']
    high = source['data']['k']['h']
    low = source['data']['k']['l']
    close = source['data']['k']['c']   
    volume = source['data']['k']['v']

    data = {'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}

    df = pd.DataFrame(data, index=[event_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

def on_message(ws, message):
    message = json.loads(message)
    extract_ohlcv(message)

In [6]:
ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:18.014  84573.53  84593.7  84494.26  84494.26  10.41565
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:20.015  84573.53  84593.7  84494.26  84494.27  10.43499
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:22.029  84573.53  84593.7  84494.26  84494.27  10.61498


True